# Transposed Convolutional Neural Networks

* The data for this notebook was obtained from J. P. Horwath, D. N. Zakharov, R. Mégret, and E. A. Stach [npj Comput. Mater.](https://doi.org/10.1038/s41524-020-00363-x) **6**, 108 (2020).

* A total of 15 transmission electron microscopy images were released as public data. Each with a resolution of $1920 \times 1792$ pixels. The atuhors state that "an ~1 nm Au film was deposited by electron beam assisted deposition in Kurt J. Lesker PVD 75 vacuum deposition system to form nanoparticles with an approximate diameter of 5 nm. The film was directly deposited onto DENSsolutions Wildfire series chips with SiN support suitable for in situ TEM heating experiments."

# Overview

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*uk4KJEtyDuPOipfG4yd-WA.gif" alt="Transposed convolution operation" align="right" style="width:300px; float:center"/>

In this notebook we will learn how to implement transposed convolution operations in neural networks. The size for these filters determines the *receptive field* of the convolution operation used to detect different features in the data. Alike conventional convolutional networks, there are three hyperparameters that control the size for the output of a transposed convolutional layer. These are related as follows

$$ O = (I - 1)\,S - 2\,P + F $$

where $O$, is the output size (height/length), $I$ is the input size (height/length), $F$ is the filter size, $P$ is the padding, and $S$ is the stride.

# Libraries

In [ ]:
import torch

import numpy as np

from pathlib          import Path
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
#
### Import local libraries
#


## 1. Prepare data

First we need to prepare our data based on the images the authors made available. The set has a shape $15 \times 1920 \times 1792$. Since these are *large* images, we will need to divide each into multiple sub-images. We have several choices, such as

* $201\,600 \times 16 \times 16$,

* $50\,400 \times 32 \times 32$,

* $12\,600 \times 64 \times 64$,

* $3\,150 \times 128 \times 128$

Since we are interested in distinguishing the deposited nano particles from the background, an image size of $128 \times 128$ is a reasonable choice to prevent loosing the features of interest. With this in mind, we need to divide all our 15 images into 3 150 sub-units, each with $128 \times 128$ pixels. For the sake of simplicity, we will give the sets **X** and **y** the generic names `images` and `labels`, respectively.

Write your choice of code in the following cell

In [ ]:
#
### Define data files
#
imhere = Path.cwd()

images = np.load(imhere/"images.npy")

labels = np.load(imhere/"labels.npy")


Before proceeding any further, we must verify that our data set actually has the shape `[3150, 128, 128]`. Both `images` and `labels` must share the same shape.

The second step is directly related to the architecture choice of the authors. The neural network they propose results two feature maps (two labels) per image, therefore we must ensure to have adjacent duplicates for each label in the set. In other words, supose our 2D labels are A, B, C, etc., as of now we have `labels = [A, B, C, ...]`, so we must turn it into `labels = [ [A, A], [B, B], [C, C], ...]`. To keep dimensions consistent, we must also add an extra dimension to the images, from `images = [A, B, C, ...]` to `images = [[A], [B], [C], ...]`.

As a result of those changes, our `images` now have a shape `[3150, 1, 128, 128]`, whereas for the `labels` we have `[3150, 2, 128, 128]`.

Write your code in the following cell

## 2. Settings and hyperparameters

Our optimization algorithm is the ADAptive Moment estimation, [Adam](https://arxiv.org/pdf/1412.6980.pdf), that is based on stochastic gradient descent.

The number of **epochs** is the number of times the learning algorithm will work through the entire training dataset.

The **batch size** is a hyperparameter that defines the number of samples to work through before updating the internal model parameters.

We will use the Cross Entropy **loss function**.

In [ ]:
#
### Training parameters
#
learnig_rate = 1e-3
weight_decay = 1e-5

epochs       = 1
batch_size   = 150
test_epoch   = 10
#
### Define neural network
#
network = # <- your object
#
### Optimizer and Loss
#
optimizer = torch.optim.Adam(params=network.parameters(), amsgrad=False, lr=learnig_rate, weight_decay=weight_decay)
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
#
### Training and testing data
#
training = DataLoader(, shuffle=True,  batch_size=batch_size) # <- insert the variable with your training data
testing  = DataLoader(, shuffle=False, batch_size=batch_size) # <- insert the variable with your testing  data

## 3. Training

We can now train our neural network for the total `epochs` we selected and testing it every `test_epoch` epochs.

Please keep in mind that passing training data to the networks consists in five steps:

1. Set the gradients to zero, `optimizer.zero_grad()`.

2. Pass batch to the network, `output = network(batch)`.

3. Compute the loss, `loss = criterion(output, y)`.

4. Perform backward pass, `loss.backward()`.

5. Perform the optimization step, `optimizer.step()`.

Remember to use the `with torch.no_grad():` context manager during testing.

Write your training and testing code in the following cell